# Homework 1: Completely Randomized Designs
# Dr. Austin R. Brown
# School of Data Science & Analytics - Kennesaw State University
# Student: Christina Talerico
# Due: Sept 12, 2025

# ------------------------------------
# Libraries
# ------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# ------------------------------------
# 1- Load the dataset- Objective of the Experiment
# ------------------------------------
df = pd.read_excel("IPA.xlsx")  # columns: AgingTime, IBU
print(df.head())

The goal is to determine whether the length of aging time (3, 4, or 5 weeks) has an effect on the bitterness of the IPA, measured in IBUs.
# ------------------------------------
#2-  Outcome Variable
# ------------------------------------
The outcome variable is the International Bitterness Units (IBUs) of the beer, a continuous measurement.  
# ------------------------------------
#3- Independent Variable + Lurking Variables
# ------------------------------------
Independent variable (factor): Aging time (3 weeks, 4 weeks, 5 weeks).
Possible lurking variables: tank-to-tank differences, storage temperature, oxygen exposure, bottle-to-bottle variation, or small errors in the spectrophotometer measurements.

# ------------------------------------
#4- Why Completely Randomized Design is Appropriate
# ------------------------------------

A completely randomized design is appropriate because all beer comes from the same initial batch, and the tanks are randomly assigned to different aging times. Randomization helps balance out any lurking factors, so differences in IBUs can be more confidently attributed to aging time.



# ------------------------------------
#5- Hypotheses
# ------------------------------------
Null hypothesis (H₀): The mean IBUs are the same across all aging times.
H0:μ=3wks=μ4wks=μ5wks

H0:μ 3wks
=μ4wks
​=μ5wks

Alternative hypothesis (Hₐ): At least one mean IBU differs among the aging times.
Ha:Atleast one μ is different
Ha:At least one μ is different

# ------------------------------------
# 6- Exploratory Analysis
# ------------------------------------
print("\nSummary statistics by aging time:")
print(df.groupby("AgingTime")["IBU"].describe())

# Boxplot
sns.boxplot(x="AgingTime", y="IBU", data=df)
plt.title("IBU Distribution by Aging Time")
plt.show()

# Mean + CI plot
sns.pointplot(x="AgingTime", y="IBU", data=df, capsize=.1, join=False)
plt.title("Mean IBU (95% CI)")
plt.show()

# ------------------------------------
# 7- Normality of residuals
# ------------------------------------
model = ols("IBU ~ C(AgingTime)", data=df).fit()
residuals = model.resid

# Q-Q plot
sm.qqplot(residuals, line='s')
plt.title("Q-Q Plot of Residuals")
plt.show()

# Shapiro-Wilk test
print("\nShapiro-Wilk test (normality):", stats.shapiro(residuals))

# ------------------------------------
# 8- Homogeneity of variance
# ------------------------------------
levene = stats.levene(
    df.loc[df["AgingTime"]==3, "IBU"],
    df.loc[df["AgingTime"]==4, "IBU"],
    df.loc[df["AgingTime"]==5, "IBU"]
)
print("\nLevene’s Test (equal variances):", levene)

# ------------------------------------
# 9- One-way ANOVA
# ------------------------------------
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nANOVA results:")
print(anova_table)

# ------------------------------------
# 10- Tukey’s HSD (only if ANOVA is significant)
# ------------------------------------
if anova_table["PR(>F)"][0] < 0.05:
    tukey = pairwise_tukeyhsd(endog=df["IBU"], groups=df["AgingTime"], alpha=0.05)
    print("\nTukey HSD results:")
    print(tukey.summary())
    tukey.plot_simultaneous()
    plt.title("Tukey HSD Pairwise Comparisons")
    plt.show()
else:
    print("\nANOVA not significant -> no Tukey test performed.")
